# QIIME2 + DADA2  Demo

Modified from Amplicons Lesson 3a created by Dr. Liz Suter as part of the BVCN (2020)

This is a demo of running QIIME2 in a JupyterLab, utilizing DADA2 to call ASVs.

This analysis replicates AstrobioMike's [amplicon analysis tutorial](https://astrobiomike.github.io/amplicon/dada2_workflow_ex#the-data) but by substituting a QIIME2 pipeline.  
_____

### General workflow

1. Initial Steps
	- Organizing your working environment
	- Checking your installations

2. Import fastq files into QIIME2
3. Remove primers
4. Check quality of trimmed reads
5. DADA2
	- Denoise
	- Generate the error model
	- Dereplicate
	- Remove chimeras
	- Merge reads
	- Infer ASVs
	- Generate the count table
6. Assign Taxonomy
7. Create a phylogenetic tree
8. Export from QIIME2 and save


_____




_____


### Initial Steps

You should be able to see the `qiime2_wd` folder in our current Binder environment, which has all of our fastq files in qiime2 format.


Make a 'work' directory where all the generated files for this analysis will go. Move this notebook to that folder as well.

In [ ]:
# mkdir work/

We will perform two different types of commands, "normal" Python commands (happen inside the Python notebook) and bash commands (using `!`) (which call the Linux command line):

In [ ]:
ls

In [ ]:
! ls

_________

### Check the QIIME2 installation

In [ ]:
! qiime info

We will also use a Python tool called qiime2. This is a pre-loaded plugin that let's us view qiime2 graphics in the notebook directly. We need to import this using python:

In [ ]:
import qiime2 as q2
# This is now a Python item (instead of bash) and doesn't need to be preceeded by `!`. We will use it a few lines down.

### Import fastq files into qiime format

In [ ]:
# Import fastq files into a qiime2 qza file format

! qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path qiime2_wd/qiime_import \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path work/demux-paired-end.qza

### Remove primers 
We want to call the same program as in Happy Belly, cutadapt, but call it from within qiime2 (documentation [here](https://github.com/qiime2/q2-cutadapt)). The [syntax](https://docs.qiime2.org/2020.2/plugins/available/cutadapt/?highlight=cutadapt) is a little different in qiime2 but we want to do the same thing. Here, we want to use the `trim-paired` method since these sequences are paired-end and already demulitplexed.

Check out how to set the parameters:

In [ ]:
! qiime cutadapt trim-paired

# How cutadapt functions for us:
From forward reads, delete forward primer from 5' end with `--p-front-f` 
and delete the reverse complement of reverse primer from 3' end with `--p-adapter-f`

From reverse reads, delete reverse primer from 5' end with `--p-front-r`
and delete the reverse complement of the forward primer from the 3' `--with p-adapter-r`

The minimum length (set with --p-minimum-length) should be 215 
According to Happy Belly, based roughly on 10% smaller than would be expected after trimming 

`--p-discard-untrimmed` states to throw away reads that don’t have the primers in them in the expected locations.

Run this as `--verbose` so we can see the output:

In [ ]:
! qiime cutadapt trim-paired \
--i-demultiplexed-sequences work/demux-paired-end.qza \
--p-cores 16 \
--p-front-f GTGCCAGCMGCCGCGGTAA \
--p-adapter-f ATTAGAWACCCBDGTAGTCC \
--p-front-r GGACTACHVGGGTWTCTAAT \
--p-adapter-r TTACCGCGGCKGCTGGCAC \
--p-minimum-length 215 \
--p-discard-untrimmed \
--o-trimmed-sequences work/demux-paired-end-trimmed.qza #\
--verbose 

You can comment out the `--verbose` option to keep the notebook clean but you can run it to see the results of primer trimming. More than 99% of the sequences get trimmed for all sample files.

### Check quality of trimmed reads
Qiime2 works with special 'qzv' files for data visualizations. To read a much better description of these, see this [page](https://docs.qiime2.org/2020.2/concepts/#data-files-visualizations). 

In [ ]:
# Use the summarize function to summarize the trimmed reads
! qiime demux summarize \
--i-data work/demux-paired-end-trimmed.qza \
--o-visualization work/demux-paired-end-trimmed.qzv

#### Visualize

<u>If you are working in your local terminal</u>, you can view the above file using the command `qiime tools view demux-paired-end-trimmed.qzv`. An html will pop up with the interactive plot. You can also drag the file directly into [this website](https://view.qiime2.org/).

<u>If you are working in the VICE app</u> in a Jupyter notebook, one of the benefits is that qiime2 visualization tool that we imported above. We can view the interactive plot directly in this notebook:

In [ ]:
ls

In [ ]:
q2.Visualization.load("demux-paired-end-trimmed.qzv")

  
  
Click around in the above interactive plot quality scores at different positions, number of sequences in different samples, etc.


From the visualization above, we can see the same things that are laid out in the Happy Belly tutorial: that, for the forward reads, the quality really drops off at about 250bp and for the reverse reads, at about 200.

### DADA2 
Use DADA2 to denoises paired-end sequences, dereplicate them, and filter chimeras, all in one step.

In the Happy Belly tutorial, DADA2 is implemented through R. This let's you control many of the parameters and is multiple steps. Within Qiime2, we just have one DADA2 step, which quality trims, generates the error model, dereplicates, removes chimeras, merges reads, infers ASVs, and generates the count table all in one step. The drawback is that here we have less control over the detailed parameters. For this example it should not impact our results, but depending on your goals, you can always go deeper.

In [ ]:
# Check the documentation on dada2 in qiime2:
!qiime dada2

In [ ]:
# We are going to use the denoise-paired option. 

# Next check which input parameters we can modfiy in that option:

!qiime dada2 denoise-paired 

Let's choose some parameters:


Goal: Replicate the parameters used in Happy Belly:

- --p-trunc-len-f: truncate the F reads at position 250 
- --p-trunc-len-r: truncate the R reads at position 200  
    - Reads shorter than these will be thrown out

- DADA2 also filters out reads using maxEE, a threshold number of expected errors in each read
    - For Happy Belly, they threw out reads if they were likely to have more than 2 erroneous base calls 
    - Here, the documentation says 2 is the default so I won't even put that parameter in. 
    - If you wanted to change it, you would use the --p-max-ee-f and --p-max-ee-r arguments

- --p-trunc-q can be used to truncate reads when the quality score drops below a certain threshold. 
    - The default number is 2, which is the value I want so I also won't be assigning that parameter         
- --p-chimera-method: I will also leave the default 'consensus' method

- --p-n-threads I am going to put 0 here, which means: use all available threads.

- --p-n-reads-learn DADA2 tries to estimate how many errors were generated by the sequencer based on an error model. This parameters sets how many sequences go into training the model. Leave the default here as 1,000,000 for now


- --verbose will show us what's happening as it's running.


**Note**: This is the longest step and may take a while.

In [ ]:
! qiime dada2 denoise-paired \
--i-demultiplexed-seqs work/demux-paired-end-trimmed.qza \
--p-trunc-len-f 250 \
--p-trunc-len-r 200 \
--p-n-threads 0 \
--output-dir work/DADA2_denoising_output \
--verbose

Let's take a look at the results 


In [ ]:
! qiime metadata tabulate \
  --m-input-file work/DADA2_denoising_output/denoising_stats.qza \
  --o-visualization work/DADA2_denoising_output/denoising_stats.qzv

In [ ]:
q2.Visualization.load("work/DADA2_denoising_output/denoising_stats.qzv")

WE can compare the above table to the `summary_tab` [table from Happy Belly](https://astrobiomike.github.io/amplicon/dada2_workflow_ex#overview-of-counts-throughout).  

Our tables look different for a number of reasons: 
* we had a few more input sequences, which was probably due to slightly different cutadapt parameters
* the filtering steps here removed several more due to the chimera removal method we used (eg. 'consensus' vs 'pool') or a parameter that we were unable to set through the qiime2 (eg. setting a min length of 175)

This highlights how much small changes can impact the result, but not the interpretation of the data. These methods all "correct" -- learning the nuances can be a long endeavor, but do not need to restrict your biological interpretations.

Take a look at a summary of the remaining sequence reads (**Note** - in the outpute of the rep_seq visualization you can click directly on a sequence to send it to Blast)

In [ ]:
! qiime feature-table tabulate-seqs \
--i-data work/DADA2_denoising_output/representative_sequences.qza \
--o-visualization work/DADA2_denoising_output/rep_seqs.qzv \

In [ ]:
q2.Visualization.load("work/DADA2_denoising_output/rep_seqs.qzv")

And take a look at the feature table

In [ ]:
! qiime feature-table summarize \
--i-table work/DADA2_denoising_output/table.qza \
--o-visualization work/DADA2_denoising_output/table.qzv \

In [ ]:
q2.Visualization.load("work/DADA2_denoising_output/table.qzv")

### Assign taxonomy 

We are going to assign taxonomy based on a reference database. These reference databases are called "classifiers" and drawn from curated databases, such as the one from [SILVA](https://www.arb-silva.de/).

You can dowload the classifier directly or train it yourself. Since the dataset we are working with uses common primers, there are classifiers that already exist.  If you are using other primers, or want to train your own reference database for some other reason, you should follow the instructions on this [page](https://docs.qiime2.org/2020.2/tutorials/feature-classifier/).

Here we are going to use  SILVA v132 as our reference database. This is the most recent qiime2-compatible classifier and is the same one used in Happy Belly. I am just going to download the 16S rRNA v4 (515F/806R) v132 classifier from [here](https://docs.qiime2.org/2021.4/data-resources/).

Next, assign taxononomy using the SILVA classifier (this step also takes awhile...)

In [ ]:
! wget https://data.qiime2.org/2019.10/common/silva-132-99-515-806-nb-classifier.qza
! mv silva-132-99-515-806-nb-classifier.qza work/

In [ ]:
! qiime feature-classifier classify-sklearn \

The cell below takes a long time to run (>1 hr)...

In [ ]:
# And try to assign taxonomy again

# Classify the representative sequences
! qiime feature-classifier classify-sklearn \
--i-classifier work/silva-132-99-515-806-nb-classifier.qza \
--i-reads work/DADA2_denoising_output/representative_sequences.qza \
--output-dir work/classified_sequences \
--verbose

In [ ]:
# and visualize
! qiime metadata tabulate \
  --m-input-file work/classified_sequences/classification.qza \
  --o-visualization work/classified_sequences/taxonomy.qzv

In [ ]:
q2.Visualization.load("work/classified_sequences/taxonomy.qzv")

### Create phylogenetic tree 

Next, we are going to make a phylogenetic tree file with our representative sequences. This is useful for some downstream analyses such as UniFrac.

In QIIME2, you can make an alignment *de novo* or align to a reference tree. To see further description of each of these, see the QIIME2 tutorial [here](https://docs.qiime2.org/2019.10/tutorials/phylogeny/).  Here, I will use the de novo approach. 

In [ ]:
# make new directory for results of alignment
! mkdir work/phylogeny

Next make a multiple sequence alignment with MAFFT ([documentation](https://docs.qiime2.org/2020.2/plugins/available/alignment/mafft/))

In [ ]:
! qiime alignment mafft \
  --i-sequences work/DADA2_denoising_output/representative_sequences.qza \
  --o-alignment work/phylogeny/aligned-rep-seqs.qza

Next you want to mask the alignment, which reduces noise from ambigously aligned regions (see qiime2 link above for more detail):

In [ ]:
! qiime alignment mask \
  --i-alignment work/phylogeny/aligned-rep-seqs.qza \
  --o-masked-alignment work/phylogeny/masked-aligned-rep-seqs.qza

Next is constructing the phylogeny. I will use fasttree here, but there are multiple options, all described in the linked QIIME2 page above.

In [ ]:
! qiime phylogeny fasttree \
  --i-alignment work/phylogeny/masked-aligned-rep-seqs.qza \
  --o-tree work/phylogeny/fasttree-tree.qza

Next, you must root the tree in order to be able to use it in UniFrac

In [ ]:
! qiime phylogeny midpoint-root \
  --i-tree work/phylogeny/fasttree-tree.qza \
  --o-rooted-tree work/phylogeny/fasttree-tree-rooted.qza

Note that all of the steps above (mafft alignment> mask> fasttree> midpoint root) could have been run with one command, `align-to-tree-mafft-fasttree`.

### Export

Next, like in Happy Belly, we want to export the major files we generated for downstream analyses: 1) the count table (aka feature table/ ASV table/ OTU table), 2) the fasta file, and 3) the taxonomy file. In addition, I will also export the tree file.


In [ ]:
! mkdir export

In [ ]:
! qiime tools export \
  --input-path work/DADA2_denoising_output/table.qza \
  --output-path work/export/table

The above file is in [BIOM format](http://biom-format.org/documentation/format_versions/biom-2.1.html). You may want to put in in tsv format is you are going to be using it in R

In [ ]:
! biom convert \
-i work/export/table/feature-table.biom \
-o work/export/table/table.tsv --to-tsv

2) Next export the fasta files with the representative sequences

In [ ]:
! qiime tools export \
  --input-path work/DADA2_denoising_output/representative_sequences.qza \
  --output-path work/export/rep-seqs.fasta

3) Export the taxonomy file

In [ ]:
! qiime tools export \
  --input-path work/classified_sequences/classification.qza \
  --output-path work/export/taxonomy

4) And lastly the tree file

In [ ]:
! qiime tools export \
  --input-path work/phylogeny/fasttree-tree-rooted.qza \
  --output-path work/export/exported-tree

### Save

And that's it! Our results files are all in the 'export' folder. You can dowload these directly to your local computer by right-clicking on the folder object on the left.